<a href="https://colab.research.google.com/github/DavidAlexandreTS/Machine-Learning-2020.2/blob/main/ML_Exercicio_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import numpy as np
import math
import random
import sys
import xlrd
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [175]:
cont_S = 0;
cont_N = 0;

In [ ]:
# Loading the Data
data = pd.read_excel('/barrettII_eyes_clustering.xlsx')
data.head()

In [169]:
# Get the location of file
plan = ("/barrettII_eyes_clustering.xlsx")
 
# To open the Workbook
wb = xlrd.open_workbook(plan)
sheet = wb.sheet_by_index(0)
 
# For row 0 and column 0
table = []
table_aux = []


i = 1;
for i in range(1, 1529):#pega 1, 2, 3 e 4
  aux = []
  #print("Eu sou o i:", i)
  j = 1
  while(j <= 5):
    aux.append(sheet.cell_value(i, j))
    #print(sheet.cell_value(i, j))
    j = j + 1

  table_aux.append(sheet.cell_value(i, j))
  table.append(aux)
  
  
#print(table_aux)


In [166]:
def UpdateMean(n, mean, item):
    for i in range(len(mean)):
        m = mean[i];
        m = (m*(n-1)+item[i])/float(n);
        mean[i] = round(m, 3);
      
    return mean;

In [142]:
def EuclideanDistance(x, y): 
    S = 0; # The sum of the squared differences of the elements 
    for i in range(len(x)): 
        S += math.pow(x[i]-y[i], 2)
  
    #The square root of the sum
    return math.sqrt(S)

In [111]:
def Classify(means,item):
  
    # Classify item to the mean with minimum distance    
    minimum = sys.maxsize;
    index = -1;
  
    for i in range(len(means)):
  
        # Find distance from item to mean
        dis = EuclideanDistance(item, means[i]);
  
        if (dis < minimum):
            minimum = dis;
            index = i;
      
    return index;

In [144]:
def StartMeans(items, k, cMin, cMax):
  
    # Initialize means to random numbers between the min and max of each column

    f = len(items[0]); # number of features
    
    means = [[0 for i in range(f)] for j in range(k)];
      
    for mean in means:
        for i in range(len(mean)):
  
            # Set value to a random float(adding +-1 to avoid a wide placement of a mean)
            mean[i] = random.uniform(cMin[i] + 1, cMax[i] - 1);
  
    return means;

In [140]:
def GetMinandMax(items):
  n = len(items[0]);
  minima = [sys.maxsize for i in range(n)];
  maxima = [-sys.maxsize -1 for i in range(n)];
	
  for item in items:
    for f in range(len(item)):
      if (item[f] < minima[f]):
        minima[f] = item[f];
      
      if (item[f] > maxima[f]):
        maxima[f] = item[f];
  
  #Those variables are lists that contain itens with
  #min and max values of all
  return minima, maxima;

In [141]:
def CalculateMeans(k, items, maxIterations=100000):

	# Find the min and max for columns
	cMin, cMax = GetMinandMax(items);
	
	# Initialize means at random points
	means = StartMeans(items, k, cMin, cMax);
	
	# Initialize clusters, the array to hold the number of items in a class
	cluSize= [0 for i in range(len(means))];

	# An array to hold the cluster an item is in
	belongsTo = [0 for i in range(len(items))];

	# Calculate means
	for e in range(maxIterations):

		# If no change of cluster occurs, halt
		noChange = True;
		for i in range(len(items)):

			item = items[i];

			# Classify item into a cluster and update the
			# corresponding means.		
			index = Classify(means,item);

			cluSize[index] += 1;
			cSize = cluSize[index];
			means[index] = UpdateMean(cSize,means[index],item);

			# Item changed cluster
			if(index != belongsTo[i]):
				noChange = False;

			belongsTo[i] = index;

		# Nothing changed, return
		if (noChange):
			break;

	return means;

In [113]:
def FindClusters(means, items):
  # Init clusters
	clusters = [[] for i in range(len(means))]; 
	
	for item in items:

		# Classify item into a cluster
		index = Classify(means,item);

		# Add item to cluster
		clusters[index].append(item);

	return clusters;


In [ ]:
means = CalculateMeans(3, table)
clusters = FindClusters(means, table)

print("Decidi dividir em 2 grupos e os indivíduos por grupo estão dispostos abaixo")
for i in range (0, 2):
  print()
  print("Grupo", i + 1,": ")
  c_aux = clusters[i]
  si = len(c_aux)
  k = 0
  while(k < si ):
    print(c_aux[k])
    k = k + 1

In [198]:
#print(clusters[0])
#qtd = len(clusters)
#print(qtd)
#qtd2 = len(table)
#qtd3 = len(table_aux)

#print(table[0])
#print(qtd2, qtd3)
global cont_S
global cont_N

frequencia = []

table_size = len(table)

for i in range (0, 2):
  c_aux = clusters[i]
  si = len(c_aux)
  k = 0
  while(k < si ):
    n = 0
    while(n < table_size):
      if table[n] == c_aux[k]:
        if table_aux[n] == "S":
          cont_S = cont_S + 1
        else:
          cont_N = cont_N + 1
      n = n + 1

    k = k + 1

  frequencia.append(cont_S)
  frequencia.append(cont_N)

  cont_S = 0
  cont_N = 0

num1_aux = frequencia[0]
num2_aux = frequencia[1]
freq_SG1 = num1_aux/(num1_aux + num2_aux)
freq_NG1 = num2_aux/(num2_aux + num1_aux)
print("Frequencia para o Grupo 1:")
print("Frequencia de SIM: ", freq_SG1, "e Frequencia de NAO: ", freq_NG1)

num11_aux = frequencia[2]
num22_aux = frequencia[3]
freq_SG2 = num11_aux/(num11_aux + num22_aux)
freq_NG2 = num22_aux/(num22_aux + num11_aux)
print("Frequencia para o Grupo 2:")
print("Frequencia de SIM: ", freq_SG2, "e Frequencia de NAO: ", freq_NG2)
#print(frequencia)
#print(num1_aux)

Frequencia para o Grupo 1:
Frequencia de SIM:  0.7255172413793104 e Frequencia de NAO:  0.27448275862068966
Frequencia para o Grupo 2:
Frequencia de SIM:  0.6987341772151898 e Frequencia de NAO:  0.3012658227848101
